In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from os.path import pardir, curdir, join

In [ ]:
df_train = pd.read_csv("../input/train.csv")
df_store = pd.read_csv("../input/store.csv")
df_test = pd.read_csv("../input/test.csv")

Features: Month, log(CompetitionDistance), Assortment, StoreType, StateHoliday & SchoolHoliday

In [ ]:
df_test['Month'] = df_test['Date'].apply(lambda x: int(x[5:7]))
df_train['Month'] = df_train['Date'].apply(lambda x: int(x[5:7]))

In [ ]:
df_store['CompetitionDistance'] = df_store['CompetitionDistance'].apply(lambda x: np.log(x))

In [ ]:
df_test['HolidayBin'] = df_test.StateHoliday.map({"0": 0, "a": 10, "b": 3, "c": 5}) * \
                        df_test.SchoolHoliday

df_train['HolidayBin'] = df_train.StateHoliday.map({"0": 0, "a": 10, "b": 3, "c": 5}).fillna(0) * \
                         df_train.SchoolHoliday
df_train.isnull().any()

In [ ]:
df_store['Assortment'] = df_store['Assortment'].map({"a": 6, "b": 8, "c": 9})

In [ ]:
df_store['StoreType'] = df_store['StoreType'].map({"a": 7, "b": 10, "c": 12, "d": 13})
df_store['StoreType']

In [ ]:
df_train['StateHoliday'] = df_train['StateHoliday'].replace(0, '0')

In [ ]:
df_test = df_test.fillna(df_test.mean())

In [ ]:
closed_store_ids = df_test["Id"][df_test["Open"] == 0].values

# remove all rows(store,date) that were closed
df_test = df_test[df_test["Open"] != 0]

In [ ]:
df_test = df_test.drop(['Date', 'StateHoliday'], axis=1)

In [ ]:
train_stores = dict(list(df_train.groupby('Store')))
test_stores = dict(list(df_test.groupby('Store')))
submission = pd.Series()
scores = []

for i in test_stores:
    store = train_stores[i]
    
    X_train = store.drop(["Date", "Sales", "Customers", "Store", "StateHoliday"], axis=1)
    Y_train = store["Sales"]
    
    X_test  = test_stores[i].copy()

    
    store_ids = X_test["Id"]
    X_test.drop(["Id","Store"], axis=1,inplace=True)
    
    lreg = linear_model.LinearRegression()
    lreg.fit(X_train, Y_train)
    
    Y_pred = lreg.predict(X_test)
    
    scores.append(lreg.score(X_train, Y_train))
    
    submission = submission.append(pd.Series(Y_pred, index=store_ids))

submission = submission.append(pd.Series(0, index=closed_store_ids))

submission = pd.DataFrame({ "Id": submission.index, "Sales": submission.values})
submission.to_csv('rossmann_submission.csv', index=False)